In [2]:
import pandas as pd

# Load the dataset
file_path = 'store_data.csv'
df = pd.read_csv(file_path)

# Display the first few rows to inspect the dataset
df.head()


,date,store_no,region,city,postal,street,longitude,latitude,store_area,location_type,...,footfall,avg_temperature,min_temperature,max_temperature,precipitation_mm,snow_depth_mm,wind_direction_degrees,wind_speed_kmh,peak_wind_gust_kmh,air_pressure_hpa
0,2021-02-12,2335,Bayern,Lauf,91207,Marktplatz 15,11.281178,49.511979,449.0,Fußgängerzone,...,NaN,-10.9,-18.0,-2.8,0.0,130.0,79.0,14.2,35.3,1033.1
1,2021-03-01,197,Rheinland-pfalz,Bitburg,54634,Trierer Str.19,6.523312,49.970275,263.0,Hauptgeschaeftsstrasse,...,NaN,6.6,0.5,14.0,0.0,NaN,73.0,13.1,NaN,1032.6
2,2021-03-01,441,Rheinland-pfalz,Annweiler,76855,Hauptstr. 35,7.963420,49.202622,350.0,Hauptgeschaeftsstrasse,...,NaN,7.4,2.4,12.8,0.0,0.0,45.0,12.5,NaN,1032.5
3,2021-03-01,632,Saarland,Lebach,66822,Am Markt 6,6.906569,49.411052,363.0,Fußgängerzone,...,3.0,5.8,0.6,12.1,0.0,0.0,74.0,13.9,41.4,1032.0
4,2021-03-01,1218,Rheinland-pfalz,Traben-Trarbach,56841,Poststr. 1 A,7.116635,49.950358,354.0,Hauptgeschaeftsstrasse,...,3.0,5.5,0.3,12.4,0.0,0.0,51.0,11.2,40.3,1033.0


In [3]:
# Get a summary of the dataset
df.info()

# Check for missing values
missing_values = df.isnull().sum()

# Display missing values
print("Missing values in each column:")
print(missing_values)

# Remove rows with missing values in 'footfall' column
df_cleaned = df.dropna(subset=['footfall'])

# Confirm the dataset size after cleaning
print(f"Dataset size after cleaning: {df_cleaned.shape}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414645 entries, 0 to 414644
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date                    414645 non-null  object 
 1   store_no                414645 non-null  int64  
 2   region                  414645 non-null  object 
 3   city                    414645 non-null  object 
 4   postal                  414645 non-null  int64  
 5   street                  414645 non-null  object 
 6   longitude               414645 non-null  float64
 7   latitude                414645 non-null  float64
 8   store_area              414645 non-null  float64
 9   location_type           414645 non-null  object 
 10  turnover                414645 non-null  float64
 11  population              414645 non-null  int64  
 12  competitor_count        414645 non-null  int64  
 13  footfall                245532 non-null  float64
 14  avg_temperature     

In [4]:
# Handle outliers by capping extreme values at the 99th percentile
for column in ['footfall', 'turnover']:
    df_cleaned[column] = df_cleaned[column].clip(upper=df_cleaned[column].quantile(0.99))

# Confirm changes
print(df_cleaned[['footfall', 'turnover']].describe())


            footfall       turnover
count  245532.000000  245532.000000
mean       19.918532     135.639404
std         8.517453      53.357485
min         2.000000      -2.032620
25%        14.000000      98.657880
50%        19.000000     130.528860
75%        25.000000     166.661445
max        47.000000     302.828209


C:\Users\amrit\AppData\Local\Temp\ipykernel_12568\927639275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[column] = df_cleaned[column].clip(upper=df_cleaned[column].quantile(0.99))
C:\Users\amrit\AppData\Local\Temp\ipykernel_12568\927639275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[column] = df_cleaned[column].clip(upper=df_cleaned[column].quantile(0.99))


In [5]:
# Add interaction feature: store_area * footfall
df_cleaned['store_area_footfall'] = df_cleaned['store_area'] * df_cleaned['footfall']

# Add month as a new feature
df_cleaned['month'] = pd.to_datetime(df_cleaned['date']).dt.month

# Group low-frequency cities into an 'Other' category
top_cities = df_cleaned['city'].value_counts().nlargest(50).index
df_cleaned['city'] = df_cleaned['city'].apply(lambda x: x if x in top_cities else 'Other')


C:\Users\amrit\AppData\Local\Temp\ipykernel_12568\25444146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['store_area_footfall'] = df_cleaned['store_area'] * df_cleaned['footfall']
C:\Users\amrit\AppData\Local\Temp\ipykernel_12568\25444146.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['month'] = pd.to_datetime(df_cleaned['date']).dt.month
C:\Users\amrit\AppData\Local\Temp\ipykernel_12568\25444146.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [6]:
# Updated feature set with new interaction terms
selected_features = [
    'store_area', 'footfall', 'avg_temperature', 'city', 
    'precipitation_mm', 'air_pressure_hpa', 'wind_speed_kmh',
    'store_area_footfall', 'month'
]
X = df_cleaned[selected_features]
y = df_cleaned['turnover']


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import xgboost as xgb

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), [
            'store_area', 'footfall', 'avg_temperature', 'precipitation_mm',
            'air_pressure_hpa', 'wind_speed_kmh', 'store_area_footfall', 'month'
        ]),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), ['city'])
    ]
)

# Define the XGBoost model
xgboost_model = xgb.XGBRegressor(random_state=42)

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgboost_model)
])


In [8]:
from sklearn.model_selection import RandomizedSearchCV

# Define hyperparameter grid
param_grid = {
    'model__n_estimators': [100, 200, 500],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [3, 6, 9],
    'model__min_child_weight': [1, 3, 5],
    'model__subsample': [0.7, 0.9, 1.0],
    'model__colsample_bytree': [0.7, 0.9, 1.0],
}

# Perform randomized search
random_search = RandomizedSearchCV(
    pipeline, param_grid, n_iter=20, cv=3, scoring='r2', random_state=42, n_jobs=-1
)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", random_search.best_params_)


c:\Users\amrit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
2 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\amrit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\amrit\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\amrit\AppData\

Best Parameters: {'model__subsample': 0.9, 'model__n_estimators': 500, 'model__min_child_weight': 1, 'model__max_depth': 9, 'model__learning_rate': 0.1, 'model__colsample_bytree': 1.0}


In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Make predictions using the best estimator
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Tuned XGBoost Model Performance:\nMAE: {mae:.2f}\nRMSE: {rmse:.2f}\nR² (Accuracy): {r2:.2f}")


Tuned XGBoost Model Performance:
MAE: 25.12
RMSE: 32.78
R² (Accuracy): 0.62
